<a href="https://colab.research.google.com/github/jayaramch743/AI-powered-REST-API-on-Spring-Boot/blob/master/multifamily_loan_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy plotly

In [ ]:
import pandas as pd
import numpy as np
import random

# Number of states to include
num_states = 10  # Reduced for easier readability

# Number of borrowers
num_borrowers = 20

# Base number of properties per state and borrower (will add some variation)
base_properties_per_state = 5
base_properties_per_borrower = 3

# List of US states (you can expand this)
states = ['California', 'Texas', 'Florida', 'New York', 'Illinois', 'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan']

# Generate random borrower names
borrower_names = [f'Borrower {i+1}' for i in range(num_borrowers)]

# Create an empty list to store the data
data = []

# Generate data for each state
for state in states:
    # Generate data for each borrower
    for borrower in borrower_names:
        # Determine the number of properties for this borrower in this state (with variation)
        num_properties = max(0, int(np.random.normal(base_properties_per_borrower, 1)))  # Ensure non-negative

        # Add data for each property
        for i in range(num_properties):
            property_value = np.random.randint(500000, 5000000) # Random property value
            data.append({
                'State': state,
                'Borrower': borrower,
                'Property_Value': property_value,
                'Units': np.random.randint(2, 20)  # Random number of units in the property
            })

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Aggregate Property Values for the same borrower on the same state
df_aggregated = df.groupby(['State', 'Borrower'])['Property_Value'].sum().reset_index()
df_aggregated = df_aggregated.rename(columns={'Property_Value': 'Total_Property_Value'})

#Add number of properties by borrower by state as well
num_properties_by_borrower_by_state = df.groupby(['State', 'Borrower'])['State'].size().reset_index(name='count')
df_aggregated = pd.merge(df_aggregated, num_properties_by_borrower_by_state, on=['State','Borrower'],how='left')

# Identify borrowers with properties in multiple states
borrower_state_counts = df['Borrower'].value_counts()
multiple_state_borrowers = borrower_state_counts[borrower_state_counts > 1].index.tolist()

# Mark borrowers who have properties in multiple states
df_aggregated['Multiple_State_Borrower'] = df_aggregated['Borrower'].apply(lambda x: x in multiple_state_borrowers)

print(df_aggregated.head()) #print the first 5 rows

        State     Borrower  Total_Property_Value  count  \
0  California   Borrower 1              11761565      4   
1  California  Borrower 10               3925777      2   
2  California  Borrower 11               8460136      3   
3  California  Borrower 12               2989589      2   
4  California  Borrower 13               9168922      2   

   Multiple_State_Borrower  
0                     True  
1                     True  
2                     True  
3                     True  
4                     True  


In [ ]:
import plotly.express as px

# Create the bubble chart
fig = px.scatter(
    df_aggregated,
    x="State",
    y="Borrower",
    size="Total_Property_Value",
    color="Multiple_State_Borrower",
    hover_data=['count'],
    title="Multiple-Unit Rental Property Distribution by State and Borrower",
    labels={"Total_Property_Value": "Total Property Value ($)", "Multiple_State_Borrower": "Multiple State Borrower"}
)

# Customize the chart
fig.update_layout(
    xaxis={'categoryorder':'total descending'}, # Sort states by total property value
    title_x=0.5,  # Center the title
    height=600,
    template="plotly_white" # A cleaner look
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# ---- "Gemmini Flash Model Code Helper" - Data Generation Module ----
def generate_rental_data(num_states=10, num_borrowers=20, base_properties_per_borrower=2, property_value_range=(500000, 2000000)):
    """
    Generates synthetic rental property data for multiple states and borrowers.
    Simulates a scenario where some borrowers manage multiple properties.

    Args:
        num_states (int): Number of US states to include in the dataset.
        num_borrowers (int): Number of unique borrowers.
        base_properties_per_borrower (int):  Average number of properties a borrower owns.  Introduce variability.
        property_value_range (tuple):  Range of property values (min, max).

    Returns:
        pandas.DataFrame: A DataFrame containing the generated data.
    """

    states = ['California', 'Texas', 'New York', 'Florida', 'Illinois', 'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan'][:num_states]  # Limiting the states
    borrower_ids = [f"Borrower_{i+1}" for i in range(num_borrowers)]

    data = []
    property_id_counter = 1  # To ensure unique property IDs

    for borrower_id in borrower_ids:
        # Introduce variability in the number of properties per borrower
        num_properties = int(np.random.normal(base_properties_per_borrower, 1))  # Mean, Std Dev.  Can result in negative values.
        num_properties = max(1, num_properties)  # Ensure at least one property

        for _ in range(num_properties):
            state = np.random.choice(states)
            property_value = np.random.randint(property_value_range[0], property_value_range[1])
            num_units = np.random.randint(2, 20)  # Number of rental units in the property

            data.append({
                'Property_ID': f"Property_{property_id_counter}",
                'State': state,
                'Borrower_ID': borrower_id,
                'Property_Value': property_value,
                'Number_of_Units': num_units
            })
            property_id_counter += 1

    df = pd.DataFrame(data)
    return df

def calculate_total_loan_amount(df):
    """
    Calculates the total loan amount for each borrower.

    Args:
        df (pandas.DataFrame): The DataFrame containing rental property data.

    Returns:
        pandas.DataFrame: The DataFrame with an added 'Total_Loan_Amount' column.
    """
    # Calculate total loan amount for each borrower
    borrower_loan_amounts = df.groupby('Borrower_ID')['Property_Value'].sum().reset_index()
    borrower_loan_amounts.rename(columns={'Property_Value': 'Total_Loan_Amount'}, inplace=True)

    # Merge total loan amounts back into the original DataFrame
    df = pd.merge(df, borrower_loan_amounts, on='Borrower_ID', how='left')
    return df

def categorize_loan_amount(amount, ranges):
    """Categorizes loan amount into predefined ranges.

    Args:
        amount (float): The loan amount to categorize.
        ranges (list of tuples): List of (lower_bound, upper_bound, category_name) tuples.

    Returns:
        str: The category name corresponding to the loan amount.
    """
    for lower, upper, category in ranges:
        if lower <= amount < upper:
            return category
    return "Other"  # Handle cases outside defined ranges


# ---- "Gemmini Flash Model Code Helper" - Visualization Module ----
def create_bubble_chart(df, size_column, color_column, hover_data_columns, title="Rental Property Distribution"):
    """
    Generates an interactive bubble chart using Plotly Express.

    Args:
        df (pandas.DataFrame): The DataFrame containing the data.
        size_column (str): Column to determine the size of the bubbles.
        color_column (str): Column to determine the color of the bubbles (categorical).
        hover_data_columns (list): List of columns to display on hover.
        title (str): Title of the chart.

    Returns:
        plotly.graph_objects.Figure: The Plotly Figure object.
    """

    fig = px.scatter(
        df,
        x="State",
        y="Borrower_ID",
        size=size_column,
        color=color_column,
        hover_data=hover_data_columns,
        title=title,
        color_discrete_sequence=px.colors.qualitative.Set1  # Optional: Set a color palette
    )

    fig.update_layout(
        xaxis_title="State",
        yaxis_title="Borrower ID",
        xaxis = dict(
            tickangle = -45
        )
    )

    return fig


# ---- Main Execution Block ----
if __name__ == "__main__":
    # 1. Data Generation
    rental_df = generate_rental_data()

    # 2. Calculate Total Loan Amount per Borrower
    rental_df = calculate_total_loan_amount(rental_df)

    # 3. Define Loan Amount Ranges (Configurable via "Gemmini Flash Model Code Helper")
    loan_amount_ranges = [
        (0, 2000000, "Under 2M"),
        (2000000, 5000000, "2M - 5M"),
        (5000000, 10000000, "5M - 10M"),
        (10000000, float('inf'), "Over 10M")
    ]

    # 4. Categorize Loan Amounts
    rental_df['Loan_Amount_Category'] = rental_df['Total_Loan_Amount'].apply(lambda x: categorize_loan_amount(x, loan_amount_ranges))

    # 5. Visualization Parameters (Configurable via "Gemmini Flash Model Code Helper")
    size_column = "Property_Value"
    color_column = "Loan_Amount_Category"  # Color by loan amount category
    hover_data_columns = ["Property_ID", "Number_of_Units", "Total_Loan_Amount"]  # Add Total Loan Amount

    # 6. Bubble Chart Creation
    fig = create_bubble_chart(rental_df, size_column, color_column, hover_data_columns)

    # 7. Display the Chart
    fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# ---- "Gemmini Flash Model Code Helper" - Data Generation Module ----
def generate_rental_data(num_states=10, num_borrowers=20, base_properties_per_borrower=2, property_value_range=(500000, 2000000)):
    """
    Generates synthetic rental property data for multiple states and borrowers.
    Simulates a scenario where some borrowers manage multiple properties.

    Args:
        num_states (int): Number of US states to include in the dataset.
        num_borrowers (int): Number of unique borrowers.
        base_properties_per_borrower (int):  Average number of properties a borrower owns.  Introduce variability.
        property_value_range (tuple):  Range of property values (min, max).

    Returns:
        pandas.DataFrame: A DataFrame containing the generated data.
    """

    states = ['California', 'Texas', 'New York', 'Florida', 'Illinois', 'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan'][:num_states]  # Limiting the states
    borrower_ids = [f"Borrower_{i+1}" for i in range(num_borrowers)]

    data = []
    property_id_counter = 1  # To ensure unique property IDs

    for borrower_id in borrower_ids:
        # Introduce variability in the number of properties per borrower
        num_properties = int(np.random.normal(base_properties_per_borrower, 1))  # Mean, Std Dev.  Can result in negative values.
        num_properties = max(1, num_properties)  # Ensure at least one property

        for _ in range(num_properties):
            state = np.random.choice(states)
            property_value = np.random.randint(property_value_range[0], property_value_range[1])
            num_units = np.random.randint(2, 20)  # Number of rental units in the property

            # Generate random economic data
            mortgage_rate = np.random.uniform(3.0, 8.0)  # Mortgage interest rate (%)
            unemployment_rate = np.random.uniform(3.0, 10.0)  # Unemployment rate (%)
            vacancy_rate = np.random.uniform(0.0, 15.0)  # Vacancy rate (%)

            data.append({
                'Property_ID': f"Property_{property_id_counter}",
                'State': state,
                'Borrower_ID': borrower_id,
                'Property_Value': property_value,
                'Number_of_Units': num_units,
                'Mortgage_Rate': mortgage_rate,
                'Unemployment_Rate': unemployment_rate,
                'Vacancy_Rate': vacancy_rate
            })
            property_id_counter += 1

    df = pd.DataFrame(data)
    return df

def calculate_total_loan_amount(df):
    """
    Calculates the total loan amount for each borrower.

    Args:
        df (pandas.DataFrame): The DataFrame containing rental property data.

    Returns:
        pandas.DataFrame: The DataFrame with an added 'Total_Loan_Amount' column.
    """
    # Calculate total loan amount for each borrower
    borrower_loan_amounts = df.groupby('Borrower_ID')['Property_Value'].sum().reset_index()
    borrower_loan_amounts.rename(columns={'Property_Value': 'Total_Loan_Amount'}, inplace=True)

    # Merge total loan amounts back into the original DataFrame
    df = pd.merge(df, borrower_loan_amounts, on='Borrower_ID', how='left')
    return df

def categorize_loan_amount(amount, ranges):
    """Categorizes loan amount into predefined ranges.

    Args:
        amount (float): The loan amount to categorize.
        ranges (list of tuples): List of (lower_bound, upper_bound, category_name) tuples.

    Returns:
        str: The category name corresponding to the loan amount.
    """
    for lower, upper, category in ranges:
        if lower <= amount < upper:
            return category
    return "Other"  # Handle cases outside defined ranges


# ---- "Gemmini Flash Model Code Helper" - Visualization Module ----
def create_bubble_chart(df, size_column, color_column, hover_data_columns, title="Rental Property Distribution"):
    """
    Generates an interactive bubble chart using Plotly Express.

    Args:
        df (pandas.DataFrame): The DataFrame containing the data.
        size_column (str): Column to determine the size of the bubbles.
        color_column (str): Column to determine the color of the bubbles (categorical).
        hover_data_columns (list): List of columns to display on hover.
        title (str): Title of the chart.

    Returns:
        plotly.graph_objects.Figure: The Plotly Figure object.
    """

    fig = px.scatter(
        df,
        x="State",
        y="Borrower_ID",
        size=size_column,
        color=color_column,
        hover_data=hover_data_columns,
        title=title,
        color_discrete_sequence=px.colors.qualitative.Set1  # Optional: Set a color palette
    )

    fig.update_layout(
        xaxis_title="State",
        yaxis_title="Borrower ID",
        xaxis = dict(
            tickangle = -45
        )
    )

    return fig


# ---- Main Execution Block ----
if __name__ == "__main__":
    # 1. Data Generation
    rental_df = generate_rental_data()

    # 2. Calculate Total Loan Amount per Borrower
    rental_df = calculate_total_loan_amount(rental_df)

    # 3. Define Loan Amount Ranges (Configurable via "Gemmini Flash Model Code Helper")
    loan_amount_ranges = [
        (0, 2000000, "Under 2M"),
        (2000000, 5000000, "2M - 5M"),
        (5000000, 10000000, "5M - 10M"),
        (10000000, float('inf'), "Over 10M")
    ]

    # 4. Categorize Loan Amounts
    rental_df['Loan_Amount_Category'] = rental_df['Total_Loan_Amount'].apply(lambda x: categorize_loan_amount(x, loan_amount_ranges))

    # 5. Visualization Parameters (Configurable via "Gemmini Flash Model Code Helper")
    size_column = "Property_Value"
    color_column = "Loan_Amount_Category"  # Color by loan amount category
    hover_data_columns = ["Property_ID", "Number_of_Units", "Total_Loan_Amount", "Mortgage_Rate", "Unemployment_Rate", "Vacancy_Rate"]  # Add economic data

    # 6. Bubble Chart Creation
    fig = create_bubble_chart(rental_df, size_column, color_column, hover_data_columns)

    # 7. Display the Chart
    fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# ---- "Gemmini Flash Model Code Helper" - Data Generation Module ----
def generate_rental_data(num_states=10, num_borrowers=20, base_properties_per_borrower=2, property_value_range=(500000, 2000000)):
    """
    Generates synthetic rental property data for multiple states and borrowers.
    Simulates a scenario where some borrowers manage multiple properties.

    Args:
        num_states (int): Number of US states to include in the dataset.
        num_borrowers (int): Number of unique borrowers.
        base_properties_per_borrower (int):  Average number of properties a borrower owns.  Introduce variability.
        property_value_range (tuple):  Range of property values (min, max).

    Returns:
        pandas.DataFrame: A DataFrame containing the generated data.
    """

    states = ['California', 'Texas', 'New York', 'Florida', 'Illinois', 'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan'][:num_states]  # Limiting the states
    borrower_ids = [f"Borrower_{i+1}" for i in range(num_borrowers)]

    data = []
    property_id_counter = 1  # To ensure unique property IDs

    for borrower_id in borrower_ids:
        # Introduce variability in the number of properties per borrower
        num_properties = int(np.random.normal(base_properties_per_borrower, 1))  # Mean, Std Dev.  Can result in negative values.
        num_properties = max(1, num_properties)  # Ensure at least one property

        for _ in range(num_properties):
            state = np.random.choice(states)
            property_value = np.random.randint(property_value_range[0], property_value_range[1])
            num_units = np.random.randint(2, 20)  # Number of rental units in the property

            # Generate random economic data
            mortgage_rate = np.random.uniform(3.0, 8.0)  # Mortgage interest rate (%)
            unemployment_rate = np.random.uniform(3.0, 10.0)  # Unemployment rate (%)
            vacancy_rate = np.random.uniform(0.0, 15.0)  # Vacancy rate (%)

            # Generate random risk data
            pd_delinquency_increase = np.random.uniform(0.0, 0.5) # probability of default to delinquency increase between 0 and 0.5

            data.append({
                'Property_ID': f"Property_{property_id_counter}",
                'State': state,
                'Borrower_ID': borrower_id,
                'Property_Value': property_value,
                'Number_of_Units': num_units,
                'Mortgage_Rate': mortgage_rate,
                'Unemployment_Rate': unemployment_rate,
                'Vacancy_Rate': vacancy_rate,
                'PD_Delinquency_Increase': pd_delinquency_increase
            })
            property_id_counter += 1

    df = pd.DataFrame(data)
    return df

def calculate_total_loan_amount(df):
    """
    Calculates the total loan amount for each borrower.

    Args:
        df (pandas.DataFrame): The DataFrame containing rental property data.

    Returns:
        pandas.DataFrame: The DataFrame with an added 'Total_Loan_Amount' column.
    """
    # Calculate total loan amount for each borrower
    borrower_loan_amounts = df.groupby('Borrower_ID')['Property_Value'].sum().reset_index()
    borrower_loan_amounts.rename(columns={'Property_Value': 'Total_Loan_Amount'}, inplace=True)

    # Merge total loan amounts back into the original DataFrame
    df = pd.merge(df, borrower_loan_amounts, on='Borrower_ID', how='left')
    return df

def categorize_loan_amount(amount, ranges):
    """Categorizes loan amount into predefined ranges.

    Args:
        amount (float): The loan amount to categorize.
        ranges (list of tuples): List of (lower_bound, upper_bound, category_name) tuples.

    Returns:
        str: The category name corresponding to the loan amount.
    """
    for lower, upper, category in ranges:
        if lower <= amount < upper:
            return category
    return "Other"  # Handle cases outside defined ranges


# ---- "Gemmini Flash Model Code Helper" - Visualization Module ----
def create_bubble_chart(df, size_column, color_column, hover_data_columns, title="Rental Property Distribution"):
    """
    Generates an interactive bubble chart using Plotly Express.

    Args:
        df (pandas.DataFrame): The DataFrame containing the data.
        size_column (str): Column to determine the size of the bubbles.
        color_column (str): Column to determine the color of the bubbles (categorical).
        hover_data_columns (list): List of columns to display on hover.
        title (str): Title of the chart.

    Returns:
        plotly.graph_objects.Figure: The Plotly Figure object.
    """

    fig = px.scatter(
        df,
        x="State",
        y="Borrower_ID",
        size=size_column,
        color=color_column,
        hover_data=hover_data_columns,
        title=title,
        color_continuous_scale=px.colors.sequential.Viridis # continuous scale for PD change
    )

    fig.update_layout(
        xaxis_title="State",
        yaxis_title="Borrower ID",
        xaxis = dict(
            tickangle = -45
        )
    )

    return fig


# ---- Main Execution Block ----
if __name__ == "__main__":
    # 1. Data Generation
    rental_df = generate_rental_data()

    # 2. Calculate Total Loan Amount per Borrower
    rental_df = calculate_total_loan_amount(rental_df)

    # 3. Define Loan Amount Ranges (Configurable via "Gemmini Flash Model Code Helper")
    loan_amount_ranges = [
        (0, 2000000, "Under 2M"),
        (2000000, 5000000, "2M - 5M"),
        (5000000, 10000000, "5M - 10M"),
        (10000000, float('inf'), "Over 10M")
    ]

    # 4. Categorize Loan Amounts
    rental_df['Loan_Amount_Category'] = rental_df['Total_Loan_Amount'].apply(lambda x: categorize_loan_amount(x, loan_amount_ranges))

    # 5. Visualization Parameters (Configurable via "Gemmini Flash Model Code Helper")
    size_column = "Property_Value"
    color_column = "PD_Delinquency_Increase"  # Color by PD Increase
    hover_data_columns = ["Property_ID", "Number_of_Units", "Total_Loan_Amount", "Mortgage_Rate", "Unemployment_Rate", "Vacancy_Rate", "PD_Delinquency_Increase"]  # Add economic data

    # 6. Bubble Chart Creation
    fig = create_bubble_chart(rental_df, size_column, color_column, hover_data_columns)

    # 7. Display the Chart
    fig.show()